<a href="https://colab.research.google.com/github/pandov/cups-mail/blob/master/segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
! sh colab.sh

In [ ]:
%%capture
import torch
import numpy as np
from src.nn import Runner, get_segmentation_components, get_loaders, score_aux
logdir = './logs/segmentation'

In [ ]:
%load_ext tensorboard
%tensorboard --logdir $logdir

In [ ]:
class MultiRunner(Runner):
    def _handle_batch(self, batch):
        x, y = batch
        y_pred = self.model(x)
        is_train = self.is_train_loader == True

        with torch.set_grad_enabled(is_train):
            loss = self.criterion(y_pred, y)
            iou = score_aux(y_pred, y)
            self.batch_metrics.update({
                'iou/_mean': np.mean(iou),
                'iou/_median': np.median(iou),
                'iou/_min': np.min(iou),
                'loss': loss,
                'lr': self.state.scheduler.get_last_lr()[0],
            })

            if is_train:
                loss.backward()
                self.optimizer.step()
                self.optimizer.zero_grad()
                if self.loader_batch_step == self.loader_batch_size:
                    self.scheduler.step()

In [ ]:
components = get_segmentation_components('unet', 'resnet50', 'adam', 'steplr')
components['loaders'] = get_loaders(keys=['image', 'mask'], batch_size=16)
runner = MultiRunner()
runner.train(
    logdir=logdir,
    num_epochs=100,
    verbose=True,
    **components
)

In [ ]:
! zip -r {logdir}.zip {logdir}
! git pull origin master
! git add logs
! git commit -m 'Changed from Colab'
! git push -u origin master